In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,70,2024-03-20,-1,122.826087,101.249275,0.577797,11.002899,23.426087,0.179232,121.113043,...,13.114706,24.655882,0.196544,111.780882,0,3,2,2,12,2024
1,69,2024-03-20,-1,109.750000,95.802941,0.535471,11.122059,21.905882,0.214176,114.379412,...,11.560294,23.580882,0.183294,116.600000,0,3,2,2,16,2024
2,69,2024-03-20,2,120.705882,99.935294,0.572279,10.864706,21.452941,0.221647,120.251471,...,10.339706,24.741176,0.189853,124.129412,0,3,2,3,17,2024
3,68,2024-03-20,1,118.044776,98.856716,0.561552,11.229851,23.819403,0.173328,118.074627,...,11.426471,24.580882,0.181118,114.436765,0,3,2,2,26,2024
4,69,2024-03-20,-2,117.191176,99.054412,0.544118,12.873529,28.686765,0.208809,117.470588,...,10.458209,20.749254,0.202194,121.191045,0,3,2,2,29,2024
5,70,2024-03-20,-3,105.884058,97.444928,0.513246,12.889855,22.760870,0.187797,108.060870,...,11.946269,27.283582,0.181910,118.385075,0,3,2,2,15,2024
6,69,2024-03-20,2,114.838235,96.985294,0.530882,10.083824,24.580882,0.229985,117.983824,...,12.583824,25.063235,0.231632,118.973529,0,3,2,2,23,2024
7,68,2024-03-20,-2,116.477612,96.516418,0.566030,11.274627,24.226866,0.216194,120.467164,...,12.704412,27.052941,0.192515,110.380882,0,3,2,3,13,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-20,IND,DET,1.227273,4.400000,1
1,2024-03-20,MIA,CLE,2.140000,1.740741,0
2,2024-03-20,MIL,BOS,4.900000,1.192308,0
3,2024-03-20,SAC,TOR,1.160000,5.550000,1
4,2024-03-20,UTA,OKC,9.000000,1.074074,0
5,2024-03-20,MEM,GSW,5.400000,1.166667,0
6,2024-03-20,PHI,PHO,4.200000,1.243902,0
7,2024-03-20,LAC,POR,1.125000,6.500000,1


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-20,IND,DET,1.227273,4.400000,1
1,2024-03-20,MIA,CLE,2.140000,1.740741,0
2,2024-03-20,MIL,BOS,4.900000,1.192308,1
3,2024-03-20,SAC,TOR,1.160000,5.550000,1
4,2024-03-20,UTA,OKC,9.000000,1.074074,0
5,2024-03-20,MEM,GSW,5.400000,1.166667,0
6,2024-03-20,PHI,PHO,4.200000,1.243902,0
7,2024-03-20,LAC,POR,1.125000,6.500000,1


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-20,IND,DET,1.227273,4.400000,1
1,2024-03-20,MIA,CLE,2.140000,1.740741,0
2,2024-03-20,MIL,BOS,4.900000,1.192308,0
3,2024-03-20,SAC,TOR,1.160000,5.550000,1
4,2024-03-20,UTA,OKC,9.000000,1.074074,0
5,2024-03-20,MEM,GSW,5.400000,1.166667,0
6,2024-03-20,PHI,PHO,4.200000,1.243902,0
7,2024-03-20,LAC,POR,1.125000,6.500000,1


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-20,IND,DET,1.227273,4.400000,1
1,2024-03-20,MIA,CLE,2.140000,1.740741,0
2,2024-03-20,MIL,BOS,4.900000,1.192308,0
3,2024-03-20,SAC,TOR,1.160000,5.550000,1
4,2024-03-20,UTA,OKC,9.000000,1.074074,0
5,2024-03-20,MEM,GSW,5.400000,1.166667,0
6,2024-03-20,PHI,PHO,4.200000,1.243902,0
7,2024-03-20,LAC,POR,1.125000,6.500000,1


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-20,IND,DET,1.227273,4.400000,1.0
1,2024-03-20,MIA,CLE,2.140000,1.740741,0.0
2,2024-03-20,MIL,BOS,4.900000,1.192308,0.0
3,2024-03-20,SAC,TOR,1.160000,5.550000,1.0
4,2024-03-20,UTA,OKC,9.000000,1.074074,0.0
5,2024-03-20,MEM,GSW,5.400000,1.166667,0.0
6,2024-03-20,PHI,PHO,4.200000,1.243902,0.0
7,2024-03-20,LAC,POR,1.125000,6.500000,1.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-20,IND,DET,1.227273,4.400000,1.0
1,2024-03-20,MIA,CLE,2.140000,1.740741,0.0
2,2024-03-20,MIL,BOS,4.900000,1.192308,0.0
3,2024-03-20,SAC,TOR,1.160000,5.550000,1.0
4,2024-03-20,UTA,OKC,9.000000,1.074074,0.0
5,2024-03-20,MEM,GSW,5.400000,1.166667,0.0
6,2024-03-20,PHI,PHO,4.200000,1.243902,0.0
7,2024-03-20,LAC,POR,1.125000,6.500000,1.0
